<a href="https://colab.research.google.com/github/Marcepan87/uDemyDA/blob/main/analiza_sentymentu_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Instalacja wymaganych bibliotek**

In [ ]:
# ---------------------------------------------
#        Instalacja wymaganych bibliotek
# ---------------------------------------------
!pip install transformers requests beautifulsoup4 pandas nltk snscrape -q
!pip install --upgrade snscrape -q
!pip install feedparser -q
!pip install python-dateutil -q
!pip install tweepy --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.9 MB/s eta 0:00:00


### **Importy**  

In [ ]:
# ---------------------------------------------
#        Importy
# ---------------------------------------------

import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import snscrape.modules.twitter as sntwitter
from transformers import pipeline

# nltk.download('stopwords')
# import nltk
# from nltk.corpus import stopwords

import time
import feedparser
from urllib.parse import quote_plus

import tweepy
from IPython.display import display, HTML





### **Model transformerowy – HerBERT**

In [ ]:
# ---------------------------------------------
#        Model transformerowy – HerBERT
# ---------------------------------------------
sentiment_pipeline = pipeline("sentiment-analysis", model="Voicelab/herbert-base-cased-sentiment")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/907k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/556k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/144 [00:00<?, ?B/s]

Device set to use cpu


###  **Funkcja czyszczenia tekstu**

In [ ]:
# ---------------------------------------------
#        Czyszczenie tekstu
# ---------------------------------------------
def clean_text(text):
    # Opcjonalnie: zamiana na małe litery
    # text = text.lower()

    # Usuwanie linków
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)

    # Usuwanie adresów email
    text = re.sub(r'\S+@\S+', '', text)

    # Usuwanie numerów telefonów (prosty wzór)
    text = re.sub(r'\b\d{9,}\b', '', text)

    # Usuwanie nadmiarowych znaków interpunkcyjnych (np. więcej niż jeden znak)
    text = re.sub(r'([!?.]){2,}', r'\1', text)

    # Usuwanie znaków specjalnych poza polskimi literami, cyframi i podstawową interpunkcją
    # Zachowujemy: a-z, ąęćłńóśźż, cyfry, spacje, przecinki, kropki, wykrzykniki, pytajniki, myślniki
    text = re.sub(r'[^A-Za-ząćęłńóśźżÓ0-9\s,.!?-]', '', text)

    # Usuwanie nadmiarowych spacji
    text = re.sub(r'\s+', ' ', text).strip()
    return text

## **Funkcja do parsowania i formatowania daty**

In [ ]:
from dateutil import parser

def normalize_date(date_str):
    try:
        dt = parser.parse(date_str, dayfirst=True)  # dayfirst=True bo polskie daty mają dzień na początku
        return dt.strftime("%d-%m-%Y, %H:%M")
    except Exception:
        return date_str  # jeśli nie uda się sparsować, zwróć oryginał

## **USTAWIENIA: Spółka i symbol**

In [ ]:
# ---------------------------------------------
#        USTAWIENIA: Spółka i symbol
# ---------------------------------------------
FIRMA = "VOTUM SA"
SYMBOL = "PKO.WA"  # dla StockTwits (opcjonalne)
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPQ91AEAAAAAPBLNWQm8er1Pciaq%2F93G5Qq%2BCgo%3Dh1GS2JiTHevFDGuYmxRGtCJ7GRagpSWE3UYzBwNcty4hUKmiLF" # pobieranie z tweetera

## **ŹRÓDŁA – Web Scraping + API**

In [ ]:
def get_bankier_articles():
    base_url = "https://www.bankier.pl"
    search_url = f"{base_url}/wyszukiwarka?qt={FIRMA.replace(' ', '+')}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'Accept-Language': 'pl-PL,pl;q=0.9',
        'Referer': base_url
    }
    r = requests.get(search_url, headers=headers)
    if r.status_code != 200:
        print(f"Błąd pobierania strony: {r.status_code}")
        return []
    soup = BeautifulSoup(r.text, 'html.parser')

    links = soup.select("article.article.hentry h2.entry-title a")
    if not links:
        print("Nie znaleziono artykułów - możliwe, że selektor jest nieaktualny.")
        return []

    articles = []
    for link in links[:7]:
        href = link.get("href")
        if not href.startswith("http"):
            href = base_url + href
        art_req = requests.get(href, headers=headers)
        if art_req.status_code != 200:
            print(f"Błąd pobierania artykułu: {art_req.status_code}")
            continue
        art_soup = BeautifulSoup(art_req.text, 'html.parser')
        paragraphs = art_soup.find_all('p')
        text = " ".join([p.get_text() for p in paragraphs])

        # Pobierz datę publikacji, np. z meta tagu lub elementu na stronie
        date_elem = art_soup.select_one("time") or art_soup.find("meta", {"property": "article:published_time"})
        if date_elem:
            if date_elem.name == "time":
                date = date_elem.get_text(strip=True)
            else:
                date = date_elem.get("content", "")
        else:
            date = ""

        if text.strip():
            articles.append({
              'źródło': 'Bankier.pl',
              'tekst': text[:1000],
              'data': date,
              'link': href
            })
    return articles


def get_google_news_rss(query=FIRMA):
    query = quote_plus(query)
    rss_url = f"https://news.google.com/rss/search?q={query}+when:7d&hl=pl&gl=PL&ceid=PL:pl"
    feed = feedparser.parse(rss_url)

    articles = []
    for entry in feed.entries[:7]:
        text = entry.title + " - " + entry.description if 'description' in entry else entry.title
        date = entry.get('published', '')  # data publikacji
        link = entry.get('link', '')  # jeśli jest link w RSS
        articles.append({
          'źródło': 'Google News RSS',
          'tekst': text,
          'data': date,
          'link': link
        })
    return articles


def get_money_articles():
    RSS_URL = "https://www.money.pl/rss/"
    USER_AGENT = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}
    max_articles = 7
    articles = []

    feed = feedparser.parse(RSS_URL)
    if feed.bozo:
        print("Błąd parsowania RSS:", feed.bozo_exception)
        return []

    for entry in feed.entries:
        title = entry.get("title", "")
        link  = entry.get("link", "")
        date  = entry.get("published", "")
        if FIRMA.lower() not in title.lower():
            continue

        try:
            r = requests.get(link, headers=USER_AGENT, timeout=5)
            if r.status_code != 200:
                continue
            soup = BeautifulSoup(r.text, "html.parser")
            ps = soup.select("div.article-text p") or soup.find_all("p")
            text = " ".join(p.get_text(strip=True) for p in ps)
            if not text:
                continue

            articles.append({
              'źródło': 'Money.pl',
              'tekst': text[:1000],
              'data': date,
              'link': link  # dodaj link
            })


            if len(articles) >= max_articles:
                break

        except Exception as e:
            print("Błąd pobierania artykułu Money.pl:", e)
            continue

    return articles

def get_pulsbiznesu_articles():
    base_url = "https://www.pb.pl"
    search_url = f"{base_url}/wyszukiwarka?q={FIRMA.replace(' ', '%20')}"
    headers = {'User-Agent': 'Mozilla/5.0'}

    r = requests.get(search_url, headers=headers)
    if r.status_code != 200:
        print(f"Błąd pobierania strony: {r.status_code}")
        return []

    soup = BeautifulSoup(r.text, 'html.parser')
    articles = []

    for a in soup.select("a > article.npb-g-t-c.npb-al-a"):
        parent_a = a.parent
        href = parent_a.get('href', '')
        if href.startswith('/'):
            href = base_url + href

        h2 = a.select_one("header.npb-a-h h2")
        title = h2.get_text(strip=True) if h2 else ''

        date_span = a.select_one("span.npb-t-c-r")
        date = date_span.get_text(strip=True) if date_span else ''

        p_lead = a.select_one("p.npb-a-l")
        lead = p_lead.get_text(strip=True) if p_lead else ''

        if title:
          text = f"{date} - {title} - {lead}"
          articles.append({
            'źródło': 'Puls Biznesu',
            'tekst': text[:1000],
            'data': date,
            'link': href  # dodaj link
          })

        if len(articles) >= 7:
            break

    return articles


def get_pap_biznes_articles():
    base_url = "https://biznes.pap.pl"
    search_url = f"{base_url}/wyszukiwarka?search={FIRMA.replace(' ', '+')}"
    headers = {'User-Agent': 'Mozilla/5.0'}

    r = requests.get(search_url, headers=headers)
    if r.status_code != 200:
        print(f"Błąd pobierania strony: {r.status_code}")
        return []

    soup = BeautifulSoup(r.text, 'html.parser')
    articles = []

    for a in soup.select("a"):
        h3 = a.select_one("h3.title")
        p_lead = a.select_one("p.field--name-field-lead")
        time_elem = a.select_one("time") or a.select_one("span.date")
        date = time_elem.get_text(strip=True) if time_elem else ''

        if h3 and p_lead:
            href = a.get('href', '')
            if href.startswith('/'):
                href = base_url + href
            title = h3.get_text(strip=True)
            lead = p_lead.get_text(strip=True)
            text = f"{date} - {title} - {lead}" if date else f"{title} - {lead}"
            articles.append({'źródło': 'PAP Biznes', 'tekst': text[:1000], 'link': href, 'data': date})
            if len(articles) >= 7:
                break

    return articles


def get_twitter_posts(max_results=20, max_return=7):
    """
    Pobiera tweety z Twitter API na podstawie globalnej zmiennej FIRMA i BEARER_TOKEN.
    Filtruje retweety, cytaty i duplikaty tekstu.
    Zwraca listę słowników z kluczami: 'źródło', 'tekst', 'data', 'link'.
    """
    global FIRMA, BEARER_TOKEN  # zakładamy, że są zdefiniowane wcześniej

    if not BEARER_TOKEN or not FIRMA:
        print("Brak BEARER_TOKEN lub FIRMA - nie można pobrać tweetów.")
        return []

    client = tweepy.Client(bearer_token=BEARER_TOKEN)

    # Budujemy zapytanie: dokładna fraza FIRMA, bez retweetów, język polski
    query = f'"{FIRMA}" -is:retweet lang:pl'

    try:
        response = client.search_recent_tweets(
            query=query,
            max_results=max_results,
            tweet_fields=['created_at', 'referenced_tweets']
        )
    except Exception as e:
        print("Błąd podczas pobierania tweetów:", e)
        return []

    if not response.data:
        return []

    filtered = []
    seen_texts = set()

    for tweet in response.data:
        # Wyklucz retweety i cytaty
        if tweet.referenced_tweets:
            if any(ref['type'] in ['retweeted', 'quoted'] for ref in tweet.referenced_tweets):
                continue
        # Filtruj duplikaty tekstu
        if tweet.text in seen_texts:
            continue
        seen_texts.add(tweet.text)

        filtered.append({
            'źródło': 'Twitter',
            'tekst': tweet.text,
            'data': tweet.created_at.isoformat(),
            'link': f"https://twitter.com/i/web/status/{tweet.id}"
        })

        if len(filtered) >= max_return:
            break

    return filtered


def get_tradingview_articles(max_items=7):
    global FIRMA  # zakładamy, że FIRMA jest zdefiniowana globalnie

    url = "https://news-mediator.tradingview.com/news-flow/v2/news?filter=lang%3Apl&client=screener&streaming=true"

    headers = {
        'User-Agent': 'Mozilla/5.0',
        'Accept': 'application/json',
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except Exception as e:
        print("Błąd pobierania lub parsowania JSON:", e)
        return []

    items = data.get("items", [])
    articles = []

    firma_lower = FIRMA.lower()

    for item in items:
        title = item.get("title", "")
        if firma_lower not in title.lower():
            continue  # pomiń artykuły, które nie zawierają FIRMA

        published_ts = item.get("published", 0)
        published_dt = datetime.utcfromtimestamp(published_ts).isoformat() if published_ts else ""
        provider = item.get("provider", {}).get("name", "")
        link = item.get("link")
        if not link:
            story_path = item.get("storyPath", "")
            link = "https://pl.tradingview.com" + story_path if story_path else ""

        articles.append({
            'źródło': provider,
            'tekst': title,
            'data': published_dt,
            'link': link
        })

        if len(articles) >= max_items:
            break

    return articles

## **Analiza wszystkich źródeł**

In [ ]:
def analyze_all_sources():
    all_posts = []
    all_posts.extend(get_bankier_articles())#
    all_posts.extend(get_google_news_rss())#
    all_posts.extend(get_money_articles())
    all_posts.extend(get_pulsbiznesu_articles())#
    all_posts.extend(get_pap_biznes_articles()) #
    all_posts.extend(get_tradingview_articles())
    # all_posts.extend(get_twitter_posts())
    # all_posts.extend(get_stocktwits_posts())

    results = []

    print(f"Liczba zebranych postów: {len(all_posts)}")
    print(f"Zebrano {len(all_posts)} postów")
    print(f"Przeanalizowano {len(results)} postów")

    for post in all_posts:
        cleaned = clean_text(post['tekst'])
        if len(cleaned.strip()) == 0:
            continue
        try:
            sentiment = sentiment_pipeline(cleaned[:512])[0]['label']
        except:
            sentiment = "Błąd"

        data_norm = normalize_date(post.get('data', ''))

        results.append({
            'Źródło': post['źródło'],
            'Tekst': post['tekst'][:250],
            'Sentyment': sentiment,
            'Data': data_norm,
            'Link': post.get('link', '')
        })

    return pd.DataFrame(results)

In [ ]:
# ---------------------------------------------
#        URUCHOMIENIE ANALIZY I SORTOWANIE
# ---------------------------------------------
df = analyze_all_sources()


# Zamień linki na klikalne linki HTML
df['Link'] = df['Link'].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>' if x else '')

# Konwersja kolumny 'Data' na datetime (jeśli jeszcze nie jest)
df['Data'] = pd.to_datetime(df['Data'], dayfirst=True, errors='coerce')

# Sortowanie po 'Źródło' rosnąco i 'Data' malejąco (najnowsze na górze)
df_sorted = df.sort_values(by=['Źródło', 'Data'], ascending=[True, False], ignore_index=True)

# Wyświetl jako HTML (w Jupyter/Colab będzie klikalne)
# display(HTML(df_sorted.to_html(escape=False)))

display(df_sorted)

Liczba zebranych postów: 28
Zebrano 28 postów
Przeanalizowano 0 postów


,Źródło,Tekst,Sentyment,Data,Link
0,Bankier.pl,1. STRONA TYTUŁOWA 2. WYBRANE DANE FINANSOWE 3...,neutral,2025-05-26 13:11:00,"<a href=""https://www.bankier.pl/wiadomosc/VOTU..."
1,Bankier.pl,1. STRONA TYTUŁOWA 2. WYBRANE DANE FINANSOWE 3...,neutral,2025-05-26 13:11:00,"<a href=""https://www.bankier.pl/wiadomosc/VOTU..."
2,Bankier.pl,1. STRONA TYTUŁOWA 2. WYBRANE DANE FINANSOWE 3...,neutral,2025-05-26 13:11:00,"<a href=""https://www.bankier.pl/wiadomosc/VOTU..."
3,Bankier.pl,1. STRONA TYTUŁOWA 2. WYBRANE DANE FINANSOWE 3...,neutral,2025-05-26 13:11:00,"<a href=""https://www.bankier.pl/wiadomosc/VOTU..."
4,Bankier.pl,1. STRONA TYTUŁOWA 2. WYBRANE DANE FINANSOWE 3...,neutral,2025-05-26 10:30:00,"<a href=""https://www.bankier.pl/wiadomosc/VOTU..."
5,Bankier.pl,1. STRONA TYTUŁOWA 2. WYBRANE DANE FINANSOWE 3...,neutral,2025-05-26 10:24:00,"<a href=""https://www.bankier.pl/wiadomosc/VOTU..."
6,Bankier.pl,„Duża uchwała frankowa” ma kompleksowo rozstrz...,positive,2024-04-22 15:09:00,"<a href=""https://www.bankier.pl/wiadomosc/Sad-..."
7,Google News RSS,8 spółek z bardzo wysoką dywidendą powyżej 8% ...,positive,2025-05-26 04:30:00,"<a href=""https://news.google.com/rss/articles/..."
8,Google News RSS,Spółka z GPW gwarantuje dywidendy w kolejnych ...,positive,2025-05-25 06:27:00,"<a href=""https://news.google.com/rss/articles/..."
9,Google News RSS,GPW podtrzymuje termin uruchomienia nowego sys...,neutral,2025-05-23 11:28:00,"<a href=""https://news.google.com/rss/articles/..."


## **PODSUMOWANIE**

In [ ]:
# ---------------------------------------------
#        PODSUMOWANIE
# ---------------------------------------------
summary = df.groupby('Źródło')['Sentyment'].value_counts().unstack().fillna(0).astype(int)
display(summary)

Sentyment,negative,neutral,positive
Źródło,,,
Bankier.pl,0,6,1
Google News RSS,0,4,3
PAP Biznes,0,5,2
Puls Biznesu,1,2,4


# testy